In [1]:
import numpy as np
import torch
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
import time

In [2]:
from src.probabilistic_dag_model.probabilistic_dag_autoencoder import ProbabilisticDAGAutoencoder
from src.probabilistic_dag_model.train_probabilistic_dag_autoencoder import train_autoencoder
from src.datasets.DAGDataset import get_dag_dataset
from src.run_probabilistic_dag_autoencoder import run


No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
Use from seml.experiment import setup_logger instead.
Note that seml.experiment.Experiment already includes the logger setup.
See https://github.com/TUM-DAML/seml/blob/master/examples/example_experiment.py


# DAG & Causal mechanisms learning from observational data

In [6]:
param_dict = {
    'seed_dataset': 123,  # Seed to shuffle dataset. int
    'dataset_name': 'Gauss_new_p50',  # Dataset name. string
    'dataset_directory': '/Users/divaselina/Library/CloudStorage/OneDrive-UniversityofCambridge/DAG/Differentiable-DAG-Sampling-master/datasets_salma', # Dataset directory. string
    'i_dataset': 1,  # Dataset name. string
    'split': [.8, .1, .1],  # Split for train/val/test sets. list of floats

    # Architecture parameters
    'seed_model': 123,  # Seed to init model. int
    'directory_model': 'saved_models',  # Path to save model. string
    'ma_hidden_dims': [16, 16, 16],  # Output dimension. int
    'ma_architecture': 'linear',  # Output dimension. int
    'ma_fast': False,  # Output dimension. int
    'pd_initial_adj': 'Learned',  # Output dimension. int
    'pd_temperature': 0.5,  # Output dimension. int
    'pd_hard': True,  # Output dimension. int
    'pd_order_type': 'topk',  # Output dimension. int
    'pd_noise_factor': 1.0,  # Hidden dimensions. list of ints

    # Training parameters
    'directory_results': '/Users/divaselina/Library/CloudStorage/OneDrive-UniversityofCambridge/DAG/Differentiable-DAG-Sampling-master/src/notebooks/saved_results',  # Path to save results. string
    'max_epochs': 100,  # Maximum number of epochs for training
    'patience': 10,  # Patience for early stopping. int
    'frequency': 2,  # Frequency for early stopping test. int
    'batch_size': 128,  # Batch size. int
    'ma_lr': 1e-4,  # Learning rate. float
    'pd_lr': 1e-3,  # Learning rate. float
    'loss': 'ELBO',  # Loss name. string
    'regr': .001,  # Regularization factor in Bayesian loss. float
    'prior_p': .001  # Regularization factor in Bayesian loss. float
}

In [7]:
results = run(**param_dict)

[02/20/25 17:47:51] INFO     Received the following configuration:          ]8;id=47805;file:///Users/divaselina/Library/CloudStorage/OneDrive-UniversityofCambridge/DAG/Differentiable-DAG-Sampling-master/src/run_probabilistic_dag_autoencoder.py\run_probabilistic_dag_autoencoder.py]8;;\:]8;id=4348;file:///Users/divaselina/Library/CloudStorage/OneDrive-UniversityofCambridge/DAG/Differentiable-DAG-Sampling-master/src/run_probabilistic_dag_autoencoder.py#64\64]8;;\

                    INFO     DATASET | seed_dataset 123 - dataset_name      ]8;id=265981;file:///Users/divaselina/Library/CloudStorage/OneDrive-UniversityofCambridge/DAG/Differentiable-DAG-Sampling-master/src/run_probabilistic_dag_autoencoder.py\run_probabilistic_dag_autoencoder.py]8;;\:]8;id=711064;file:///Users/divaselina/Library/CloudStorage/OneDrive-UniversityofCambridge/DAG/Differentiable-DAG-Sampling-master/src/run_probabilistic_dag_autoencoder.py#65\65]8;;\
                             Gauss_new_p50 - i_dataset 1 - split [0.8, 0.1,                                        
                             0.1]                                                                                  

                    INFO     ARCHITECTURE |  seed_model 123 -               ]8;id=343866;file:///Users/divaselina/Library/CloudStorage/OneDrive-UniversityofCambridge/DAG/Differentiable-DAG-Sampling-master/src/run_probabilistic_dag_autoencoder.py\run_probabilistic_dag_autoencoder.py]8;;\:]8;id=683795;file:///Users/divaselina/Library/CloudStorage/OneDrive-UniversityofCambridge/DAG/Differentiable-DAG-Sampling-master/src/run_probabilistic_dag_autoencoder.py#70\70]8;;\
                             ma_hidden_dims [16, 16, 16] -  ma_architecture                                        
                             linear -  ma_fast False -  pd_initial_adj                                             
                             Learned -  pd_temperature 0.5 -  pd_hard True                                         
                             -  pd_order_type topk -  pd_noise_factor 1.0                                          

                    INFO     TRAINING |  max_epochs 100 -  patience 10 -    ]8;id=35111;file:///Users/divaselina/Library/CloudStorage/OneDrive-UniversityofCambridge/DAG/Differentiable-DAG-Sampling-master/src/run_probabilistic_dag_autoencoder.py\run_probabilistic_dag_autoencoder.py]8;;\:]8;id=448325;file:///Users/divaselina/Library/CloudStorage/OneDrive-UniversityofCambridge/DAG/Differentiable-DAG-Sampling-master/src/run_probabilistic_dag_autoencoder.py#80\80]8;;\
                             frequency 2 -  batch_size 128 -  ma_lr 0.0001                                         
                             -  pd_lr 0.001 -  loss ELBO -  regr 0.001 -                                           
                             prior_p 0.001                                                                         

X: tensor([[ 1.8930, -0.4186,  1.7951,  ...,  2.3407, -1.1721,  0.6902],
        [-0.3705,  0.8874,  1.1301,  ...,  2.6210,  1.2884,  2.3754],
        [ 1.1297,  1.3914,  1.6137,  ...,  0.8433, -0.5540,  2.4492],
        ...,
        [ 2.0685,  0.6073,  1.2963,  ...,  1.8180,  0.7765, -0.7371],
        [ 0.9009,  0.8757,  0.3007,  ...,  0.9537,  1.7218,  0.5356],
        [-0.2041,  1.3590, -1.4315,  ...,  1.0072,  0.7875, -0.9320]]), X_pred:tensor([[0.7714, 0.7466, 0.6419,  ..., 0.8436, 0.6994, 0.7546],
        [0.7886, 0.7468, 0.6405,  ..., 0.8530, 0.7118, 0.7508],
        [0.7709, 0.7539, 0.6518,  ..., 0.8553, 0.7075, 0.7471],
        ...,
        [0.7787, 0.7479, 0.6405,  ..., 0.8504, 0.7035, 0.7302],
        [0.7567, 0.7500, 0.6414,  ..., 0.8538, 0.7011, 0.7578],
        [0.7807, 0.7474, 0.6466,  ..., 0.8535, 0.7080, 0.7719]],
       grad_fn=<SoftplusBackward0>)
first term: 1.4202057123184204
second term: 0.03386890888214111
total ELBO: 1.4540746212005615
X: tensor([[ 0.4759,  0.12

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


X: tensor([[ 2.0939, -0.1026,  1.0420,  ...,  1.6604,  0.2694,  1.5224],
        [ 0.8653,  1.4119,  1.4732,  ..., -0.4138,  0.4441,  0.9371],
        [ 0.9169,  0.4130,  2.0402,  ...,  2.1257, -1.9920, -0.3732],
        ...,
        [ 1.2032,  0.7876,  2.0857,  ...,  1.0654,  0.5095,  1.8280],
        [-0.2692, -0.6508,  1.5130,  ...,  0.7635, -0.3049, -0.1383],
        [ 0.0862, -1.2659,  0.0408,  ...,  1.5209,  0.2041, -1.9082]]), X_pred:tensor([[0.8241, 0.9201, 0.7615,  ..., 1.0332, 0.6414, 0.6855],
        [0.8241, 0.9201, 0.7615,  ..., 1.0332, 0.6414, 0.6855],
        [0.8241, 0.9201, 0.7615,  ..., 1.0332, 0.6414, 0.6855],
        ...,
        [0.8241, 0.9201, 0.7615,  ..., 1.0332, 0.6414, 0.6855],
        [0.8241, 0.9201, 0.7615,  ..., 1.0332, 0.6414, 0.6855],
        [0.8241, 0.9201, 0.7615,  ..., 1.0332, 0.6414, 0.6855]])
first term: 1.381272792816162
second term: 0.02145099639892578
total ELBO: 1.402723789215088
